In [82]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', None)

plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline
pd.set_option('display.max_columns', None)

In [83]:
data_string=r"C:\Users\am513\PS-88-21-DEV\lab\aaronfolder\NHIS2009_clean.dta" 
df_og=pd.read_stata(data_string, convert_missing=True)
df=pd.read_stata(data_string)
df;
#df.to_csv("NHIS2009_clean.csv")

In [168]:
df_og2=df_og.query("marradult == 1 & perweight != 0")
df_og2

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,hhweight,pernum,perweight,sampweight,age,marstat,sex,famsize,relate,racenew,educ,educrec1,empstat,incfam07on,health,uninsured,age2,fml,nwhite,hi,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1
1,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,1,8938.0,22029.0,29,Married,Female,4 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$0 - $34,999",Very Good,Not covered,841.0,1.0,0.0,0.0,14,0.0,4.0,19282.931641,17500.0,1.0,1.0,0.0,1.0,.
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,4,8967.0,0.0,35,Married,Male,4 people,Spouse,White,Grade 11,Grade 11,Working for pay at job/business,"$0 - $34,999",Very Good,Not covered,1225.0,0.0,0.0,0.0,11,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0
7,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,1,8905.0,0.0,32,Married,Male,4 people,Householder,White,High school graduate,Grade 12,Working for pay at job/business,"$100,000 and over",Good,Covered,1024.0,0.0,0.0,1.0,12,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1
8,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,2,8889.0,22190.0,34,Married,Female,4 people,Spouse,White,"Bachelor's degree (BA, AB, BS, BBA)",4 years of college/Bachelor's degree,Working for pay at job/business,"$100,000 and over",Good,Covered,1156.0,1.0,0.0,1.0,16,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,.
11,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,7871,1,8378.0,19284.0,65,Married,Male,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$35,000 - $49,999",Good,Covered,4225.0,0.0,0.0,1.0,14,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80620,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,2220,2,2496.0,5732.0,57,Married,Female,2 people,Spouse,White,High school graduate,Grade 12,Not in labor force,"$100,000 and over",Very Good,Covered,3249.0,1.0,0.0,1.0,12,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,.
80624,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,1,3135.0,7200.0,67,Married,Female,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Good,Covered,4489.0,1.0,0.0,1.0,14,0.0,3.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,.
80625,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,2,3022.0,0.0,68,Married,Male,2 people,Spouse,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Fair,Covered,4624.0,0.0,0.0,1.0,14,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1
80626,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,2200,1,2532.0,18062.0,62,Married,Male,7 people,Spouse,White,Grade 9,Grade 9,Working for pay at job/business,"$100,000 and over",Poor,Not covered,3844.0,0.0,0.0,0.0,9,1.0,1.0,167844.531250,150000.0,0.0,1.0,1.0,2.0,0


In [106]:
df2=df.query("marradult == 1 & perweight != 0") #keep if marradult==1 & perweight!=0 
df2;

In [170]:
df2.groupby("serial").sum();#.reset_index()[["serial","hi_hsb1"]]

In [108]:
df4=df2.merge(df3,how="left",on="serial")
df4;

In [109]:
df5=df4.drop("hi_hsb1_x",axis=1).rename(columns={"hi_hsb1_y":"hi_hsb1"})

In [156]:
df5_5=df5[["serial","hi_hsb1"]].set_index("serial")
df5_5 #by serial: egen hi_hsb = mean(hi_hsb1)

,hi_hsb1
serial,
3,0.0
3,0.0
5,1.0
5,1.0
6,1.0
...,...
41173,1.0
41175,1.0
41175,1.0


In [164]:
df_og3=df_og2.drop("hi_hsb1",axis=1).set_index("serial") #joining the cleaned hi_hsb values to the main df

df_og3["hi_hsb"]=df5_5["hi_hsb1"]
df_og4=df_og3.reset_index()

In [167]:
df_og4.dropna(subset=["hi_hsb","hi"]).head(50) #na values get converted to 0 :(

,serial,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,hhweight,pernum,perweight,sampweight,age,marstat,sex,famsize,relate,racenew,educ,educrec1,empstat,incfam07on,health,uninsured,age2,fml,nwhite,hi,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb
0,3,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,1,8938.0,22029.0,29,Married,Female,4 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$0 - $34,999",Very Good,Not covered,841.0,1.0,0.0,0.0,14,0.0,4.0,19282.931641,17500.0,1.0,1.0,0.0,1.0,0.0
1,3,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,4,8967.0,0.0,35,Married,Male,4 people,Spouse,White,Grade 11,Grade 11,Working for pay at job/business,"$0 - $34,999",Very Good,Not covered,1225.0,0.0,0.0,0.0,11,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0
2,5,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,1,8905.0,0.0,32,Married,Male,4 people,Householder,White,High school graduate,Grade 12,Working for pay at job/business,"$100,000 and over",Good,Covered,1024.0,0.0,0.0,1.0,12,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0
3,5,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,2,8889.0,22190.0,34,Married,Female,4 people,Spouse,White,"Bachelor's degree (BA, AB, BS, BBA)",4 years of college/Bachelor's degree,Working for pay at job/business,"$100,000 and over",Good,Covered,1156.0,1.0,0.0,1.0,16,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0
4,6,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,1,8378.0,19284.0,65,Married,Male,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$35,000 - $49,999",Good,Covered,4225.0,0.0,0.0,1.0,14,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0
5,6,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,2,9015.0,0.0,66,Married,Female,2 people,Spouse,White,"Some college, no degree",1 to 3 years of college,Working for pay at job/business,"$35,000 - $49,999",Good,Covered,4356.0,1.0,0.0,1.0,14,1.0,3.0,41679.343750,42500.0,0.0,1.0,1.0,1.0,1.0
6,7,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,1,7741.0,21587.0,64,Married,Male,2 people,Householder,White,High school graduate,Grade 12,Not in labor force,"$35,000 - $49,999",Poor,Covered,4096.0,0.0,0.0,1.0,12,0.0,1.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0
7,7,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7871,2,8558.0,0.0,59,Married,Female,2 people,Spouse,White,GED or equivalent,Grade 12,Working for pay at job/business,"$35,000 - $49,999",Fair,Covered,3481.0,1.0,0.0,1.0,12,1.0,2.0,41679.343750,42500.0,1.0,1.0,1.0,1.0,1.0
8,8,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,8658,1,8817.0,0.0,60,Married,Male,4 people,Householder,White,"Bachelor's degree (BA, AB, BS, BBA)",4 years of college/Bachelor's degree,Not in labor force,"$100,000 and over",Very Good,Covered,3600.0,0.0,0.0,1.0,16,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,1.0
9,8,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,8658,2,9746.0,0.0,52,Married,Female,4 people,Spouse,White,"Some college, no degree",1 to 3 years of college,Working for pay at job/business,"$100,000 and over",Very Good,Covered,2704.0,1.0,0.0,1.0,14,1.0,4.0,167844.531250,150000.0,1.0,1.0,1.0,1.0,1.0
